In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
 #sklearn preproc

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error

import pickle

In [3]:
cpountry_list=['Russia','China']
countries=[]

In [4]:
for country in country_list:
    path=f'../data/out_csv/index_{country}.csv'
    df=pd.read_csv(path)
    countries=[{'name':country,'df':df}]
countries[0]['name']

NameError: name 'country_list' is not defined

In [ ]:
path1='/home/sumedha/code/covid_time_series_prediction/covid_time_series_prediction/data/models'

In [ ]:
best_params=[]
best_score=[]
best_df:{}
for country in countries:
    df=country['name']
    df=country['df']
    df=df.set_index('date')
    df['day-1']=df['total_deaths'].shift(periods=1)
    df['day-2']=df['total_deaths'].shift(periods=2)
    df['day-3']=df['total_deaths'].shift(periods=3)
    df['day-4']=df['total_deaths'].shift(periods=4)
    df['day-5']=df['total_deaths'].shift(periods=5)
    df['day-6']=df['total_deaths'].shift(periods=6)
    df['day-7']=df['total_deaths'].shift(periods=7)
    df['day-8']=df['total_deaths'].shift(periods=8)
    df['day-9']=df['total_deaths'].shift(periods=9)
    df['day-10']=df['total_deaths'].shift(periods=10)
    df = df.iloc[10: , :]
    df=df.fillna(0)
    X=df.drop(columns=['total_deaths','new_deaths','new_cases'])
    y=df['total_deaths']
    scaler = MinMaxScaler()
    scaler.fit(X)
    X=scaler.transform(X)
    n = len(X)
    X_train = X[0:int(n-50)]
    X_test=X[int(n-50):]
    y_train=y[0:int(n-50)]
    y_test=y[int(n-50):]
    
    model=SVR()
    param={'kernel' : ('poly', 'rbf'),'C' : [5,6],'degree' : [3,8],'coef0' : [0.01,0.5,10]}
    grid_search = GridSearchCV(model, param_grid = param, cv = 2, n_jobs = -1, verbose = 2)
    grid_search.fit(X_train,y_train)
    best=grid_search.best_estimator_
    model=best
    best_model=model.fit(X_train,y_train)
    name=country['name']
    with open(f'{path1}/model_{name}.pkl','wb') as f:pickle.dump(best_model, f)
    
    y_pred=best_model.predict(X_test)
    score=mean_absolute_percentage_error(y_test, y_pred)
    best_params.append(grid_search.best_params_)
    best_score.append(score)
    best_df = pd.DataFrame({
    'country': f'data_{name}',
    'best_params': best_params,
    'best_score': best_score
})
    
    

## prediction function